<a href="https://colab.research.google.com/github/suribe1010/Kaggle/blob/master/Transfer_Learning_Example_for_NLP_ULMFiT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q torch==1.4.0 torchvision

In [3]:
!pip install fastai


In [0]:
import fastai
from fastai import *
from fastai.text import *
import pandas as pd
import numpy as np
from functools import partial
import io
import os


In [5]:
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers','footers', 'quotes'))

In [0]:
documents=dataset.data

In [7]:
df = pd.DataFrame({'label': dataset.target, 'text': dataset.data})
df

,label,text
0,17,Well i'm not sure about the story nad it did s...
1,0,"\n\n\n\n\n\n\nYeah, do you expect people to re..."
2,17,Although I realize that principle is not one o...
3,11,Notwithstanding all the legitimate fuss about ...
4,10,"Well, I will have to change the scoring on my ..."
...,...,...
11309,17,"Danny Rubenstein, an Israeli journalist, will ..."
11310,13,\n
11311,9,\nI agree. Home runs off Clemens are always m...
11312,4,I used HP DeskJet with Orange Micros Grappler ...


In [8]:
df = df[df['label'].isin([1,10])] 
df

,label,text
4,10,"Well, I will have to change the scoring on my ..."
10,1,Archive-name: graphics/resources-list/part1\nL...
17,10,"\nAnd of course, Mike Ramsey was (at one time)..."
24,10,"As I promised, I would give you the name of th..."
28,10,GAME(S) OF 4/15\n---------------\nADIRONDACK 6...
...,...,...
11248,10,The Hawks win!! Jermey Roenick scored his 50 ...
11259,10,I think that NHLPA' 93 is the best video game ...
11267,1,\nI am in the market for a 24-bit graphics car...
11288,1,"Hi there,\n\nis there anybody who know a polyg..."


In [9]:
df = df.reset_index(drop = True)
df

,label,text
0,10,"Well, I will have to change the scoring on my ..."
1,1,Archive-name: graphics/resources-list/part1\nL...
2,10,"\nAnd of course, Mike Ramsey was (at one time)..."
3,10,"As I promised, I would give you the name of th..."
4,10,GAME(S) OF 4/15\n---------------\nADIRONDACK 6...
...,...,...
1179,10,The Hawks win!! Jermey Roenick scored his 50 ...
1180,10,I think that NHLPA' 93 is the best video game ...
1181,1,\nI am in the market for a 24-bit graphics car...
1182,1,"Hi there,\n\nis there anybody who know a polyg..."


In [10]:
df['label'].value_counts()

10    600
1     584
Name: label, dtype: int64

In [12]:
# Data Preprocessing
df['text'] = df['text'].str.replace("[^a-zA-Z]"," ")
df

,label,text
0,10,Well I will have to change the scoring on my ...
1,1,Archive name graphics resources list part La...
2,10,And of course Mike Ramsey was at one time ...
3,10,As I promised I would give you the name of th...
4,10,GAME S OF ADIRONDACK C...
...,...,...
1179,10,The Hawks win Jermey Roenick scored his ...
1180,10,I think that NHLPA is the best video game ...
1181,1,I am in the market for a bit graphics card...
1182,1,Hi there is there anybody who know a polygon...


In [19]:
# Get rid of stopwords
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

# Tokenization
tokenized_doc = df['text'].apply(lambda x: x.split())
tokenized_doc

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0       [Well, I, will, have, to, change, the, scoring...
1       [Archive, name, graphics, resources, list, par...
2       [And, of, course, Mike, Ramsey, was, at, one, ...
3       [As, I, promised, I, would, give, you, the, na...
4       [GAME, S, OF, ADIRONDACK, CDI, Adirondack, lea...
                              ...                        
1179    [The, Hawks, win, Jermey, Roenick, scored, his...
1180    [I, think, that, NHLPA, is, the, best, video, ...
1181    [I, am, in, the, market, for, a, bit, graphics...
1182    [Hi, there, is, there, anybody, who, know, a, ...
1183    [You, might, want, to, clarify, the, game, win...
Name: text, Length: 1184, dtype: object

In [25]:
# Remove stop words
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
tokenized_doc

0       [Well, I, change, scoring, playoff, pool, Unfo...
1       [Archive, name, graphics, resources, list, par...
2       [And, course, Mike, Ramsey, one, time, captain...
3       [As, I, promised, I, would, give, name, Panthe...
4       [GAME, S, OF, ADIRONDACK, CDI, Adirondack, lea...
                              ...                        
1179    [The, Hawks, win, Jermey, Roenick, scored, th,...
1180    [I, think, NHLPA, best, video, game, available...
1181    [I, market, bit, graphics, card, PC, ISA, bus,...
1182    [Hi, anybody, know, polygon, reduction, algori...
1183    [You, might, want, clarify, game, winning, str...
Name: text, Length: 1184, dtype: object

In [0]:
# De-tokenization
